# CuRobo Reaching Demo
This notebook should be run with a kernel that has the CuRobo installed as instructed [here](). The goal is to compute the jointspace motion required for reaching a target pose in the robot workspace. Then this motion is to be tracked using the FR3 joint velocity interface.

In [1]:
from FR3Py.robot.interface import FR3Real

In [2]:
robot = FR3Real(robot_id="fr3")

Interface Running...


In [3]:
robot.getStates()

{'q': array([ 0.06120732, -0.64873199,  0.10074113, -2.19144209,  0.05109697,
         1.5524614 ,  0.90973614,  0.        ,  0.        ]),
 'dq': array([-6.90158624e-04,  7.51744422e-04, -1.06131301e-03, -6.08870503e-05,
        -3.44473922e-04,  1.14973473e-03,  4.08892038e-04,  0.00000000e+00,
         0.00000000e+00]),
 'T': array([-0.26254582, -7.51947165, -2.67592454, 23.3774147 ,  1.4128288 ,
         2.22104621,  0.19915226,  0.        ,  0.        ])}

In [28]:

import numpy as np
import threading
import time

class JointPositionController:
    def __init__(self, robot, dT=0.001):
        self.dT = dT
        self.robot = robot
        self.kp = 0.4
        self.kv = 0.01
        state = self.robot.getStates()
        if state is not None:
            self.q0 = state['q'][:7]
        else:
            raise ValueError("Failed to get robot state. Check the interface.")
        self.q_des = self.q0[:7]
        self.dq_des = np.zeros(7)
        self.running = True
        self.thread = threading.Thread(target=self.update)
        self.thread.start()
    
    def update(self):
        while self.running:
            time.sleep(self.dT)
            state = self.robot.getStates()
            if state is not None:
                q = state['q'][:7]
                dq = state['dq'][:7]
                dq_cmd = self.kp * (self.q_des - q) + self.kv*(self.dq_des-dq) + self.dq_des
                self.robot.sendCommands(np.hstack([dq_cmd, 0.0, 0.0]))
            else:
                print("Failed to get robot state. Check the interface.")

    def close(self):
        self.running = False
        self.thread.join()

In [29]:
controller = JointPositionController(robot)

In [9]:
# cuRobo
from curobo.geom.sdf.world import CollisionCheckerType
from curobo.types.base import TensorDeviceType
from curobo.types.math import Pose
from curobo.types.robot import JointState, RobotConfig
from curobo.util_file import (
    get_robot_configs_path,
    get_world_configs_path,
    join_path,
    load_yaml,
    )
from curobo.wrap.reacher.motion_gen import MotionGen, MotionGenConfig, MotionGenPlanConfig

In [10]:
from curobo.geom.types import WorldConfig, Cuboid, Mesh, Capsule, Cylinder, Sphere
from curobo.util_file import get_assets_path, join_path

world_cfg_table = WorldConfig.from_dict(
    load_yaml(join_path(get_world_configs_path(), "collision_table.yml"))
)
world_cfg_table.cuboid[0].pose[2] -= 0.02
world_cfg1 = WorldConfig.from_dict(
    load_yaml(join_path(get_world_configs_path(), "collision_table.yml"))
).get_mesh_world()
world_cfg1.mesh[0].name += "_mesh"
world_cfg1.mesh[0].pose[2] = -10.5

world_cfg = WorldConfig(cuboid=world_cfg_table.cuboid, mesh=world_cfg1.mesh)

In [11]:
trajopt_dt = None
optimize_dt = True
trajopt_tsteps = 32
trim_steps = None
max_attempts = 4
interpolation_dt = 0.05
trajopt_tsteps = 40
trajopt_dt = 0.05
optimize_dt = False
max_attempts = 1
trim_steps = [1, None]
interpolation_dt = trajopt_dt
n_obstacle_cuboids = 30
n_obstacle_mesh = 100

In [12]:
tensor_args = TensorDeviceType()
world_file = "collision_table.yml"
robot_file = "franka.yml"
robot_cfg = load_yaml(join_path(get_robot_configs_path(), robot_file))["robot_cfg"]
robot_cfg = RobotConfig.from_dict(robot_cfg, tensor_args)
motion_gen_config = MotionGenConfig.load_from_robot_config(
        robot_cfg,
        world_cfg,
        tensor_args,
        collision_checker_type=CollisionCheckerType.MESH,
        num_trajopt_seeds=12,
        num_graph_seeds=12,
        interpolation_dt=interpolation_dt,
        collision_cache={"obb": n_obstacle_cuboids, "mesh": n_obstacle_mesh},
        optimize_dt=optimize_dt,
        trajopt_dt=trajopt_dt,
        trajopt_tsteps=trajopt_tsteps,
        trim_steps=trim_steps,
        # velocity_scale=0.1,
        # self_collision_check=False,
        # self_collision_opt=False,
    )
motion_gen = MotionGen(motion_gen_config)
print("warming up...")
motion_gen.warmup(enable_graph=True, warmup_js_trajopt=False, parallel_finetune=True)

warming up...


True

In [54]:
import torch
import numpy as np
q = robot.getStates()['q'][:7]
current_state = motion_gen.rollout_fn.compute_kinematics(
    JointState.from_position(torch.from_numpy(q.astype(np.float32)).view(1, -1).cuda()))

target_pose = Pose(current_state.ee_pos_seq.squeeze()-0.2, quaternion=current_state.ee_quat_seq.squeeze())


start_state = JointState.from_position(torch.from_numpy(q.astype(np.float32)).view(1, -1).cuda())

result = motion_gen.plan_single(
    start_state, target_pose, MotionGenPlanConfig(max_attempts=1)
)
print(result.success.item())

print(result.optimized_plan.position.shape)
traj = result.get_interpolated_plan()
print("Trajectory Generated: ", result.success, result.optimized_dt.item())

True
torch.Size([39, 7])
Trajectory Generated:  tensor([True], device='cuda:0') 0.05000000074505806


In [55]:
dq_des = result.optimized_plan.velocity.detach().cpu().numpy()
q_des  = result.optimized_plan.position.detach().cpu().numpy()
for i in range(q_des.shape[0]):
    controller.q_des = q_des[i]
    controller.dq_des = dq_des[i]
    time.sleep(result.optimized_dt.item())

In [51]:
controller.kp = 0.4
controller.kv = 0.01

In [56]:
controller.close()